In [18]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpDot
from utilities import print_result

### Simple Linear Optimization

PuLP can solve the linear optimization problems in the following format.
$$
\min {(3x - 5y)}
$$
$$s.t.
y \leq x 
$$

Here, $3x-5y$ is called the objective function. In this problem, we are trying to minimize this function, while making sure the *constraints* hold. In this problem, there is only one constraint, $y \leq x$.

In [7]:
# problem definition with name and 'sense'
problem = LpProblem('Simple Minimization', LpMinimize)

In [8]:
# Variables are defined with names, bounds, and categories
x = LpVariable('x', lowBound=0, cat='Continuous')
y = LpVariable('y', upBound=5, cat='Integer')

In [9]:
# the objective is added to the problem as a statement
problem += 3 * x - 5 * y

In [10]:
# the constraints are added as conditionals
problem += y <= x

In [11]:
problem.solve()

# problem status is 1 if optimized
print_result(problem)

Optimization status: 1
Final value of the objective: -10.0
Final values of the variables:
x = 5.0
y = 5.0


### Declaring a List of Variables

If there are multiple variables which have the same category, lower and upper bounds, it is possible to use a dictionary and list comprehensions to declare variables, objective, and constraints.

Consider a classic problem of maximixing utility with cost limitations.
$$
\max \sum_{i=1}^4 x_i u_i
$$
$$
s.t. \sum_{i=1}^4 x_i c_i \leq 1
$$
$$
0\leq x_i \leq 20
$$

In [12]:
# A list of variables suffixes (x1, x2, x3, x4)
variable_names = [1, 2, 3, 4]

# Cost of each variables is defined in a dictionary
costs = {
    1: 0.05,
    2: 0.02,
    3: 0.07,
    4: 0.09
}

# Similarly utilities are defined in a dictionary
utilities = {
    1: 10,
    2: 8,
    3: 15,
    4: 20
}

In [14]:
# Variables are defined as a dictionary with comman bounds and category
# variables is a dictionary with keys being the elements of the variable_names list
# Values of this dict is the variable names (x_1, x_2...) which can be referred to define objectives and constraints
variables = LpVariable.dict('x', variable_names, lowBound=0, upBound=20, cat='Continuous')

In [15]:
problem = LpProblem('Maximize Utility', LpMaximize)

In [16]:
# Objective is defined as a dot product of the variables and corresponding utilities
problem += lpDot(variables.values(), utilities.values())

# Similarly, total cost is defined as the dot product of variables and costs
problem += lpDot(variables.values(), costs.values()) <= 1

In [17]:
problem.solve()

print_result(problem)

Optimization status: 1
Final value of the objective: 293.33333400000004
Final values of the variables:
x_1 = 0.0
x_2 = 20.0
x_3 = 0.0
x_4 = 6.6666667
